# Example of cart pole simulation in Mujoco
*****************************************************************
 Copyright © 2025 [Wei Zhang/CLEAR Lab]. All Rights Reserved.

********************************************************************
This tutorial introduces how to set up closed-loop control system for a simple cart pole systems in Mujoco. 

First, let's create the cart pole model

In [97]:
import mujoco
import mediapy as media
import numpy as np

import mujoco.viewer as viewer
modelxml = """
<mujoco model="CartPole">
  <compiler eulerseq="XYZ"/>
  <default>
    <default class="unused"/>
  </default>
  <asset>
    <texture name="grid" type="2d" builtin="checker" rgb1="0.1 0.2 0.3" rgb2="0.2 0.3 0.4" width="512" height="512"/>
    <material name="grid" class="unused" texture="grid" texrepeat="1 1" texuniform="true" reflectance="0.2"/>
  </asset>
  <worldbody>
    <geom name="floor" class="unused" type="plane" condim="3" size="0 0 0.05" material="grid" pos="0 0 -1"/>
    <body name="Cart" pos="0 0 0" euler="0 -0 0">
    <!-- change inertia, different from sdf -->
    <!-- For this model case, with the cart not having any rotational
             degrees of freedom, the values of the inertia matrix do not
             participate in the model. Therefore we just set them to zero
             (or near to zero since sdformat does not allow exact zeroes
             for inertia values). -->
      <inertial pos="0 0 0" mass="10" diaginertia="0.00000000001 0.00000000001 0.00000000001"/>
      <geom name="cart_visual" class="unused" type="box" contype="0" conaffinity="0" group="0" size="0.12 0.06 0.06" pos="0 0 0" euler="0 -0 0"/>
      <joint name="CartSlider" class="unused" type="slide" pos="0 0 0" axis="1 0 0"/>
      <body name="Pole" pos="0 0 -0.5" euler="0 -0 0">
        <inertial pos="0 0 0" mass="1" diaginertia="0.00000000001 0.00000000001 0.00000000001"/>
        <geom name="pole_point_mass" class="unused" type="sphere" contype="0" conaffinity="0" group="0" size="0.05" pos="0 0 0" euler="0 -0 0"/>
        <geom name="pole_rod" class="unused" type="cylinder" contype="0" conaffinity="0" group="0" size="0.025 0.25" pos="0 0 0.25" euler="0 -0 0"/>
        <joint name="PolePin" class="unused" type="hinge" pos="0 0 0.5" axis="0 -1 0"/>
      </body>
    </body>
  </worldbody>
  <sensor>
        <!-- joint position sensing -->
        <jointpos joint="CartSlider" name="cart_p"/>
        <jointpos joint="PolePin" name="pole_theta"/>
        <!-- joint velocity sensing -->
        <jointvel joint="CartSlider" name="cart_v"/>
        <jointvel joint="PolePin" name="pole_w"/>
  </sensor>
  <actuator>
    <motor joint="CartSlider"/>
  </actuator>
  <keyframe>
    <key name="off1" qpos="0 3.15"/>
  </keyframe>
</mujoco>
"""
model = mujoco.MjModel.from_xml_string(modelxml)
data = mujoco.MjData(model)


## define LQR controller
- 增加角度$x_2$的权重会使控制器更积极地校正杆的角度
- 增加位置$x_1$的权重会限制小车的移动范围
- 增加速度项$(x_3,x_4)$的权重会使系统响应更平滑
- 减小R值会使控制器使用更大的控制力，响应更快但可能不稳定
- 增大R值会使控制更温和，但可能无法有效稳定系统

In [ ]:
# define customized controller which returns the feedback control action
# you can implement your controller here 

# 以下为原文件示例代码
# def myControl(model, data):
#     x = np.hstack((data.qpos,data.qvel))
#     xref = np.array([0, np.pi, 0, 0])
#     x_error = x-xref
#     K = np.array([-0.5, 0.5, 0, 0])
#     u = K@x_error
#     data.ctrl = u
#     return u

from scipy.linalg import solve_continuous_are

def setup_lqr_k():
    # 系统矩阵（来自proj1.ipynb的线性化结果）
    A = np.array([
        [0, 0, 1, 0],
        [0, 0, 0, -1],
        [0, -0.981, 0, 0],
        [0, -21.582, 0, 0]
    ])
    B = np.array([[0], [0], [0.1], [0.2]])
    
    Q = np.diag([10, 100000, 0.0001, 0.0001])
    R = np.array([[0.0000001]])
    
    P = solve_continuous_are(A, B, Q, R)
    K = np.linalg.inv(R) @ B.T @ P
    
    return K

def lqr_controller(model, data):
    # 获取当前状态
    x = np.array([
        data.qpos[0],        # 小车位置x1
        data.qpos[1] - np.pi, # 杆角度x2（相对于竖直向上位置）
        data.qvel[0],        # 小车速度x3
        data.qvel[1]         # 杆角速度x4
    ])

    K = setup_lqr_k()
    u = K @ x
    
    # 将控制输入限制在合理范围内
    u = np.clip(u, -20, 20)
    
    return u[0]  # 返回标量控制输入

## Simulate the closed-loop system

In [99]:
import mujoco.viewer as viewer
import time

mujoco.mj_resetDataKeyframe(model, data, 0)  # Reset the state to keyframe 0 （默认的初始状态）

# 以下注释部分为手动调节初始状态
# mujoco.mj_resetData(model, data)
# data.qpos[0] = 0.0  # 小车初始位置
# data.qpos[1] = np.pi+0.01  # 杆初始角度(π弧度=180度)
# data.qvel[0] = 0.0  # 小车初始速度
# data.qvel[1] = 0.0  # 杆初始角速度

# 
with viewer.launch_passive(model, data) as viewer:  
  # launch_passive means all the simulation should be done by the user 
  
  start = time.time()
  while viewer.is_running() and time.time() - start < 20:
    step_start = time.time()
    # data.ctrl = 0.2*myControl(model,data)
    data.ctrl = lqr_controller(model, data)
    mujoco.mj_step(model, data)

  # let viewer show updated info
    viewer.sync()
    
  # #  make sure the while loop is called every sampling period 
    # computation inside the loop may take some nontrivial time. 
    time_until_next_step = model.opt.timestep - (time.time() - step_start)
    if time_until_next_step > 0:
      time.sleep(time_until_next_step)